In [1]:
!pip install --no-deps --quiet nltk rank_bm25 sentence-transformers tqdm

import json
import nltk
import os, heapq
import numpy as np
import torch

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus    import stopwords
from nltk.util      import everygrams
from transformers   import pipeline
from rank_bm25      import BM25Okapi
from sentence_transformers import SentenceTransformer, util
from tqdm.auto     import tqdm
from typing         import List

2025-05-30 10:12:04.590975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748599924.761352      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748599924.815549      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# NLTK assets
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")

# stopwords
stops = set(stopwords.words("english"))
stops.update(["would","could"])
# append space to match original logic (optional)
stops = {w + " " for w in stops}
STOPWORDS= stops

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
import json

#path = "/kaggle/input/documentzz"
file = json.load(open("/kaggle/input/documentzz/documentzz.json", "r", encoding = 'utf-8'))
docs = file
file

[[17.741650576283966,
  'Wikipedia',
  'for the winter olympics in st moritz switzerland total of eight sports venues were used the five venues used for the winter olympics were reused for these games three new venues were added for alpine skiing which had been added to the winter olympics program twelve years earlier in garmisch partenkirchen germany allied occupied germany during the games as of the bob run continues to be used for bobsleigh and the cresta run for skeleton while alpine skiing remains popular in st moritz venues venue sports capacity ref around the hills of st moritz cross country skiing nordic combined cross country skiing not listed cresta run skeleton not listed kulm ice hockey not listed olympiaschanze st moritz ski jumping nordic combined ski jumping not listed olympic stadium opening closing ceremonies figure skating ice hockey final speed skating not listed piz nair alpine skiing not listed st moritz celerina olympic bobrun bobsleigh not listed suvretta ice hoc

In [4]:
# NER pipeline
ner = pipeline(
    "ner",
    model="dslim/bert-large-NER",
    tokenizer="dslim/bert-large-NER",
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

def clean_query(query: str,
                remove_stopwords: bool = True,
                remove_qmark:    bool = True) -> str:
    if remove_qmark and query.endswith("?"):
        query = query[:-1]
    if remove_stopwords:
        tokens = []
        for w in query.split():
            if (w.lower() + " ") not in STOPWORDS:
                tokens.append(w)
        query = " ".join(tokens)
    return query

def Gen_Ngram(tokens: List[str], max_ngram: int = None) -> List[str]:
    """Produce all ngrams up to length=`max_ngram` (default = len(tokens))."""
    L = len(tokens)
    n = max_ngram or L
    n = min(n, L)
    ngs = everygrams(tokens, max_len=n)
    return [" ".join(gram) for gram in ngs]

def generate_multiview_queries(question: str, max_ngram: int = 4) -> List[str]:
    """
    1) Clean question (drop '?' and stopwords)
    2) NER sub-queries
    3) N-grams (1..max_ngram)
    4) Noun-only single query
    """
    # (A) Clean
    q_clean = clean_query(question)

    # (B) Named Entities
    ner_ents = [ent["word"] for ent in ner(q_clean)]
    
    # (C) N-grams
    toks     = [t for t in question.replace("?","").split() if t.isalpha()]
    ngrams   = Gen_Ngram(toks, max_ngram)
    
    # (D) Nouns
    pos_tags = nltk.pos_tag(nltk.word_tokenize(question.replace("?","")))
    nouns    = [w for w,tag in pos_tags if tag in {"NN","NNS","NNP","NNPS"}]
    
    # merge & dedupe, drop length-1
    all_q = set(ner_ents + ngrams + nouns)
    return [q for q in all_q if len(q.split())>1]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Path to the dump
DUMP_PATH = "/kaggle/input/wikidumps/wikipedia-dump.txt"

In [6]:
import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    print(dirname, filenames)


/kaggle/input []
/kaggle/input/wikidumps ['wikipedia-dump.txt']
/kaggle/input/documentzz ['documentzz.json']
/kaggle/input/sentence ['sents.json']
/kaggle/input/develop ['dev.json']
/kaggle/input/training ['train.jsonl']
/kaggle/input/document ['docs.json']


# Topic Retriever

In [7]:
# ───BM25 Topic Retriever Helpers ─────────────────────────────────────
import heapq

def _process_chunk_multiview(paras, titles, queries, top_d, heap):
    """
    Tokenize a list of paragraphs, build BM25, score all `queries`,
    and keep the top_d entries in `heap`.
    """
    
    tokenized = [
        [w for w in word_tokenize(p.lower()) if w.isalpha() and w not in stops]
        for p in paras
    ]
    # 2) Build BM25 index
    bm25 = BM25Okapi(tokenized)
    # 3) Score: for each query, compute BM25 scores and take elementwise max
    scores = np.zeros(len(paras), dtype=float)
    for q in queries:
        q_tok   = [w for w in word_tokenize(q.lower()) if w.isalpha()]
        scores  = np.maximum(scores, bm25.get_scores(q_tok))
    # 4) Maintain a min-heap of size top_d
    for sc, t, p in zip(scores, titles, paras):
        if len(heap) < top_d:
            heapq.heappush(heap, (sc, t, p))
        elif sc > heap[0][0]:
            heapq.heapreplace(heap, (sc, t, p))

In [8]:
import json


#dump_path = "/kaggle/input/wikidumps/wikipedia-dump.txt"
def iter_wiki_paragraphs(dump_path: str):
    """
    Plain text dump: one paragraph per line.
    Yields (title, paragraph) for each non‐empty line ≥ 50 chars.
    """
    with open(dump_path, "r", encoding="utf-8") as f:
        for lineno, line in enumerate(f, start=1):
            p = line.strip()
            if len(p) < 50:
                continue
            yield "wikipedia", p

In [9]:
def local_topic_retrieve_mv(
    question: str,
    top_d: int = 5,
    chunk_size: int = 50_000
) -> list[tuple[float,str,str]]:
    """
    Multiview BM25 over your wiki dump (one paragraph per line).
    Returns the top_d (score, title, paragraph) tuples.
    """
    # 1) Generate sub-queries from the question
    queries = generate_multiview_queries(question)

    # 2) Stream through the dump in chunks
    heap, buffer, titles = [], [], []
    for title, para in tqdm(iter_wiki_paragraphs(DUMP_PATH), desc="Topic retrieve"):
        buffer.append(para)
        titles.append(title)
        if len(buffer) >= chunk_size:
            _process_chunk_multiview(buffer, titles, queries, top_d, heap)
            buffer, titles = [], []

    # 3) Process any leftovers
    if buffer:
        _process_chunk_multiview(buffer, titles, queries, top_d, heap)

    # 4) Extract and sort the top scoring docs
    return sorted(heap, key=lambda x: -x[0])[:top_d]

In [10]:
# ─── Cell: Download & Verify NLTK Perceptron Tagger Resources ─────────────────

import nltk
from nltk import pos_tag, word_tokenize

# 1) Download JSON‐based English Perceptron Tagger
nltk.download("averaged_perceptron_tagger_eng")

# 2) Also ensure the standard (Pickle) tagger is present
nltk.download("averaged_perceptron_tagger")

# 3) Quick smoke-test to confirm it loads correctly
tokens = word_tokenize("NLTK POS tagging now works without error.")
tagged  = pos_tag(tokens)
print("POS tags:", tagged)


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


POS tags: [('NLTK', 'NNP'), ('POS', 'NNP'), ('tagging', 'VBG'), ('now', 'RB'), ('works', 'VBZ'), ('without', 'IN'), ('error', 'NN'), ('.', '.')]


## sanity check

In [11]:
from itertools import islice

sample = list(islice(iter_wiki_paragraphs(DUMP_PATH), 5))
print("First 5 paragraphs:", sample)


First 5 paragraphs: [('wikipedia', 'anarchism is political philosophy and movement that rejects all involuntary coercive forms of hierarchy it calls for the abolition of the state which it holds to be undesirable unnecessary and harmful it is usually described alongside libertarian marxism as the libertarian wing libertarian socialism of the socialist movement and as having historical association with anti capitalism and socialism the history of anarchism goes back to prehistory when some humans lived in anarchistic societies long before the establishment of formal states realms or empires with the rise of organised hierarchical bodies skepticism toward authority also rose but it was not until the th century that self conscious political movement emerged during the latter half of the th and the first decades of the th century the anarchist movement flourished in most parts of the world and had significant role in workers struggles for emancipation various anarchist schools of thought f

# docs calulated

In [ ]:
# ─── Test Local‐dump Topic Retriever ───────────────────────────────
question = "What is the ruling party of the country that hosted the 2016 Olympics?"
docs = local_topic_retrieve_mv(question, top_d=50, chunk_size=50_000)

print("Top 50 paragraphs from local dump:")
for sc, t, p in docs:
    print(f"{sc:.1f} → {t}: {p[:100]}…")

print(f"Retrieved {len(docs)} paragraphs.")


save the docs 

In [12]:
#docs

# Attribute retriever

In [13]:
import torch
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util
from typing import List, Tuple

# 1) Ensure stopwords & device
nltk.download("stopwords")
stops = set(stopwords.words("english"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def attribute_retrieve(question, docs, top_k=50):
    from sentence_transformers import SentenceTransformer, util
    import torch
    import re

    # Load encoder
    attr_enc = SentenceTransformer("paraphrase-mpnet-base-v2", device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    # Extract sentences from paragraphs
    sents = []
    for _, _, para in docs:
        sents.extend(re.split(r"(?<=[.?!])\s+", para.strip()))

    # Masked version of the question
    q_masked = re.sub(r"\b(?:who|what|when|where|why|how|which)\b", "[MASK]", question, flags=re.I)

    # Mask the sentences (same as question)
    masked = [re.sub(r"\b(?:who|what|when|where|why|how|which)\b", "[MASK]", s, flags=re.I) for s in sents]

    # Assign device properly
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Encode both question and sentences
    q_vec  = attr_enc.encode([q_masked], convert_to_tensor=True, device=device)     # shape: (1, 384)
    s_vecs = attr_enc.encode(masked,    convert_to_tensor=True, device=device)      # shape: (N, 384)

    # Compute cosine similarity
    cos_scores = util.cos_sim(q_vec, s_vecs)[0]
    top_idxs = torch.topk(cos_scores, k=min(top_k, len(sents))).indices

    return [(sents[i], cos_scores[i].item()) for i in top_idxs]


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Test Attribute Retriever 

In [15]:
question = "What is the ruling party of the country that hosted the 2016 Olympics?"
#docs     = local_topic_retrieve_mv(question, top_d=50, chunk_size=50_000)
top_sents= attribute_retrieve(question, docs, top_k=50)

print("Top 5 sentences:")
for sent, sc in top_sents:
    print(f"{sc:.3f} → {sent}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Top 5 sentences:
0.515 → there were six bids initially submitted for the summer olympics tokyo was ultimately elected as the host city at the th ioc session in buenos aires argentina on september bidding process the olympic bidding process begins with the submission of city application to the international olympic committee ioc by its national olympic committee noc and ends with the election of the host city by the members of the ioc during an ordinary session the process is governed by the olympic charter as stated in chapter rule since the process has consisted of two phases during the first phase which begins immediately after the bid submission deadline the applicant cities are required to answer questionnaire covering themes of importance to successful games organization this information allows the ioc to analyze the cities hosting capacities and the strengths and weaknesses of their plans following detailed study of the submitted questionnaires and ensuing reports the ioc executi

#  Save Results for Download 

In [16]:
import os, json, shutil
from IPython.display import FileLink

# 1) Ensure working cache dir
WORKING_CACHE = "/kaggle/working/cache"
os.makedirs(WORKING_CACHE, exist_ok=True)

# 2) Paths
docs_path  = os.path.join(WORKING_CACHE, "docs.json")
sents_path = os.path.join(WORKING_CACHE, "sents.json")

# 3) Save docs & sents
with open(docs_path,  "w", encoding="utf-8") as f:
    json.dump([{"score":s,"title":t,"para":p} for s,t,p in docs],
              f, ensure_ascii=False, indent=2)
with open(sents_path, "w", encoding="utf-8") as f:
    json.dump([{"sent":s,"score":sc} for s,sc in top_sents],
              f, ensure_ascii=False, indent=2)

print(f"→ Saved {len(docs)} docs to  {docs_path}")
print(f"→ Saved {len(top_sents)} sents to {sents_path}")

# 4) (Optional) Zip the entire cache folder
zip_name = "/kaggle/working/retrieval_cache"
shutil.make_archive(zip_name, 'zip', WORKING_CACHE)
zip_path = zip_name + ".zip"
print(f"→ Bundled into {zip_path}")

# 5) Display download links
display(FileLink(docs_path))
display(FileLink(sents_path))
display(FileLink(zip_path))

→ Saved 50 docs to  /kaggle/working/cache/docs.json
→ Saved 50 sents to /kaggle/working/cache/sents.json
→ Bundled into /kaggle/working/retrieval_cache.zip


/kaggle/working/cache/docs.json

/kaggle/working/cache/sents.json

/kaggle/working/retrieval_cache.zip

cached retrieved results and sentences

# **Disintigrated reasoning**

In [17]:
# ─── Cell 0: Configuration ────────────────────────────────────────────────────
from dataclasses import dataclass, field
from typing      import List
import os

@dataclass
class Config:
    # Paths
    train_path: str       = "./data/train_gdsent.json"
    dev_path:   str       = "./data/dev_gdsent.json"
    test_path:  str       = "./data/test.json"
    pretrained_model_path: str = "./pretrained_model/6_STAR_ORA-P/weights.th"
    model_checkpoint_path:  str = "./checkpoints/combination/1.pth"
    wikipedia_dump: str   = "/kaggle/input/wikidumps/wikipedia-dump.txt"

    # Hyperparameters
    epoch_num:    int     = 20
    batch_size:   int     = 1
    learning_rate: float  = 1e-5
    tuning_rate:  float   = 0.4   # the ι weight for strategy loss
    weight_decay: float   = 0.01
    max_length:   int     = 512
    num_workers:  int     = 2

    def __post_init__(self):
        os.makedirs(os.path.dirname(self.model_checkpoint_path), exist_ok=True)

# Create the global config
cfg = Config()


In [18]:
docs

[[17.741650576283966,
  'Wikipedia',
  'for the winter olympics in st moritz switzerland total of eight sports venues were used the five venues used for the winter olympics were reused for these games three new venues were added for alpine skiing which had been added to the winter olympics program twelve years earlier in garmisch partenkirchen germany allied occupied germany during the games as of the bob run continues to be used for bobsleigh and the cresta run for skeleton while alpine skiing remains popular in st moritz venues venue sports capacity ref around the hills of st moritz cross country skiing nordic combined cross country skiing not listed cresta run skeleton not listed kulm ice hockey not listed olympiaschanze st moritz ski jumping nordic combined ski jumping not listed olympic stadium opening closing ceremonies figure skating ice hockey final speed skating not listed piz nair alpine skiing not listed st moritz celerina olympic bobrun bobsleigh not listed suvretta ice hoc

# saving the docs


import json

with open('documentzz.json', 'w', encoding='utf-8') as f:
            json.dump(docs, f) # indent=2 for a bit more compact pretty print
print(f"JSON downloaded and saved to '{docs}'")

In [19]:
import json

#path = "/kaggle/input/documentzz"
file = json.load(open("/kaggle/input/documentzz/documentzz.json", "r", encoding = 'utf-8'))
docs = file
file

[[17.741650576283966,
  'Wikipedia',
  'for the winter olympics in st moritz switzerland total of eight sports venues were used the five venues used for the winter olympics were reused for these games three new venues were added for alpine skiing which had been added to the winter olympics program twelve years earlier in garmisch partenkirchen germany allied occupied germany during the games as of the bob run continues to be used for bobsleigh and the cresta run for skeleton while alpine skiing remains popular in st moritz venues venue sports capacity ref around the hills of st moritz cross country skiing nordic combined cross country skiing not listed cresta run skeleton not listed kulm ice hockey not listed olympiaschanze st moritz ski jumping nordic combined ski jumping not listed olympic stadium opening closing ceremonies figure skating ice hockey final speed skating not listed piz nair alpine skiing not listed st moritz celerina olympic bobrun bobsleigh not listed suvretta ice hoc

# StrategyQA

# **get new code here

In [20]:
!pip install transformers scikit-learn


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Dataset Class
class StrategyQADataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=128):
        with open(file_path, "r") as f:
            self.data = [json.loads(line) for line in f]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item["question"]
        facts = item.get("facts", "")
        answer = int(item["answer"])  # True -> 1, False -> 0

        encoding = self.tokenizer(
            question,
            facts,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {key: val.squeeze(0) for key, val in encoding.items()}, answer

# Model Class
class StrategyQAModel(nn.Module):
    def __init__(self):
        super(StrategyQAModel, self).__init__()
        self.encoder = RobertaModel.from_pretrained("roberta-base")
        self.classifier = nn.Linear(self.encoder.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        return self.classifier(cls_output)

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = StrategyQAModel()
model.load_state_dict(torch.load("model.pt", map_location="cpu"))  # Path to your trained model
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
# Load dev set
dev_dataset = StrategyQADataset("/kaggle/input/strategyqa/dev.json", tokenizer)
dev_loader = DataLoader(dev_dataset, batch_size=16)

# Run evaluation
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in tqdm(dev_loader):
        inputs, labels = batch
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = torch.tensor(labels).to(device)

        logits = model(**inputs)
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

# Report accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"✅ Dev Accuracy: {accuracy * 100:.2f}%")


In [ ]:
def compute_recall_at_k(retrieved_dict, gold_dict, k=10):
    """
    Computes Recall@k for retrieval results.

    Args:
        retrieved_dict (dict): question_id → list of retrieved passage strings (ranked)
        gold_dict (dict): question_id → list of gold passage strings
        k (int): number of top retrieved passages to consider (e.g., 10)

    Returns:
        float: recall@k score
    """
    assert set(retrieved_dict.keys()) == set(gold_dict.keys()), "Mismatch in question IDs"

    num_questions = len(gold_dict)
    num_hit = 0

    for qid in gold_dict:
        top_k_passages = retrieved_dict[qid][:k]
        gold_passages = gold_dict[qid]

        # Normalize passages if needed
        top_k_set = set([p.strip().lower() for p in top_k_passages])
        gold_set = set([g.strip().lower() for g in gold_passages])

        # Check if at least one gold passage is in the top-k
        if top_k_set & gold_set:
            num_hit += 1

    recall_at_k = num_hit / num_questions
    return recall_at_k


# gold evidence

In [ ]:
import json

# Load StrategyQA data
with open("path_to/strategyqa_train.json", "r") as f:
    data = json.load(f)

# Extract gold evidence per question
gold_passages = {}
for item in data:
    qid = item["qid"]
    evidence_sets = item.get("evidence", [])
    gold_set = set()
    for evidences in evidence_sets:
        if isinstance(evidences, list):
            for ev in evidences:
                gold_set.add(ev.strip().lower())
    gold_passages[qid] = list(gold_set)


In [ ]:
# Assume saved as: {"qid1": ["para1", ..., "para10"], ...}
with open("path_to/retrieved_paragraphs.json", "r") as f:
    retrieved_passages = json.load(f)


# recall@10

In [ ]:
def recall_at_k(gold_dict, retrieved_dict, k=10):
    total = 0
    hit = 0

    for qid, gold_list in gold_dict.items():
        gold_set = set(gold_list)
        retrieved_list = retrieved_dict.get(qid, [])[:k]
        retrieved_set = set(p.strip().lower() for p in retrieved_list)

        if gold_set & retrieved_set:
            hit += 1
        total += 1

    recall = hit / total if total else 0
    print(f"Recall@{k}: {recall:.4f} ({hit}/{total})")

recall_at_k(gold_passages, retrieved_passages, k=10)


In [ ]:
import json
import nltk
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from tqdm import tqdm

nltk.download("punkt")

# Load StrategyQA questions
with open("path_to/strategyqa_train.json", "r") as f:
    qa_data = json.load(f)

# Load corpus of documents
# Example corpus: dict with {"doc_id": "text"}
with open("path_to/corpus.json", "r") as f:
    corpus = json.load(f)

# Tokenize corpus
corpus_ids = list(corpus.keys())
corpus_texts = [corpus[doc_id] for doc_id in corpus_ids]
tokenized_corpus = [word_tokenize(doc.lower()) for doc in corpus_texts]

# Build BM25 index
bm25 = BM25Okapi(tokenized_corpus)

# Retrieve top-10 paragraphs per question
retrieved = {}

for ex in tqdm(qa_data, desc="Retrieving"):
    qid = ex["qid"]
    question = ex["question"]
    query_tokens = word_tokenize(question.lower())

    scores = bm25.get_scores(query_tokens)
    topk_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:10]
    topk_passages = [corpus_texts[i] for i in topk_indices]

    retrieved[qid] = topk_passages

# Save to JSON
with open("retrieved_passages.json", "w") as f:
    json.dump(retrieved, f, indent=2)


# Test‐set Inference & Save Predictions

In [ ]:
import json
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report

# 1) Load the best checkpoint
model.load_state_dict(torch.load(cfg.model_checkpoint_path, map_location=device))
model.eval()

# 2) Load StrategyQA test split
test_ds = load_dataset("ChilleD/StrategyQA", split="test")  # fields: question, answer, strategy

all_gold, all_pred = [], []
outputs = []

for ex in test_ds:
    q    = ex["question"]
    gold = bool(ex["answer"])
    strat= ex["strategy"]

    # a) Retrieval & Attr (uses your cached helpers)
    docs    = load_or_compute_docs(q, top_d=5, chunk_size=50_000)
    sents   = load_or_compute_sents(q, docs, top_k=5)

    # b) Inference
    strat_pred, ans_prob = infer_example(q, docs, sents)
    pred_ans = ans_prob > 0.5

    # c) Collect for metrics & save
    all_gold.append(int(gold))
    all_pred.append(int(pred_ans))

    outputs.append({
        "question":      q,
        "gold_answer":   int(gold),
        "pred_answer":   int(pred_ans),
        "answer_prob":   ans_prob,
        "gold_strategy": strat,
        "pred_strategy": STRAT_LABELS[strat_pred]
    })

# 3) Compute & print metrics
acc = accuracy_score(all_gold, all_pred)
print(f"\nTest Accuracy: {acc:.3%}\n")
print(classification_report(all_gold, all_pred, target_names=["No","Yes"]))

# 4) Save detailed predictions
out_path = f"{cfg.output_dir}/strategyqa_test_predictions.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(outputs, f, ensure_ascii=False, indent=2)
print(f"✅ Saved all predictions to {out_path}")


# analysing failed predictions

In [ ]:
import json
from collections import Counter

# 1.1) Load the saved test predictions
preds = json.load(open(f"{cfg.output_dir}/strategyqa_test_predictions.json", "r"))

# 1.2) Compute per-strategy breakdown
by_strat = Counter((ex["gold_strategy"], ex["pred_strategy"]) for ex in preds)
print("Confusion counts (gold → pred):")
for (g,p),cnt in by_strat.items():
    print(f"  {g:10s} → {p:10s}: {cnt}")

# 1.3) List some failure cases for spot-checking
failures = [ex for ex in preds if ex["gold_answer"] != ex["pred_answer"]]
print(f"\nTotal failures: {len(failures)} / {len(preds)}")
for ex in failures[:5]:
    print(f"\nQ: {ex['question']}\nGold: {ex['gold_answer']}, Pred: {ex['pred_answer']}")
    print("Evidence sentences:")
    docs = load_or_compute_docs(ex["question"])
    sents = load_or_compute_sents(ex["question"], docs)
    for s,_ in sents:
        print("  -", s)


In [ ]:
import time

count = 0
while True:
    print(count)
    count += 1
    
    print("Waiting for 2 minutes...")
    time.sleep(120)
    print("Resuming execution now.")
       
